In [1]:
# import the dependencies
## install dependencies in terminal as follows
# pip install numpy
# pip install pandas
# pip install xgboost
# pip install -U scikit-learn

import numpy as np
import pandas as pd 

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

In [13]:
# load the dataset

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv', names = train.columns)
nans = test.pop('popularity')
y = train.pop('popularity')
X = train

In [14]:
# train:test splits 80:20 ratio

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2)

In [67]:
# analysis for the correct classifiers for the prediction

names = ["Nearest Neighbors", "RBF SVM","xgb"]

classifiers = [
    KNeighborsClassifier(6, leaf_size=25, weights='distance'),
    SVC(gamma=0.4, C=7, decision_function_shape='ovr'),
    xgb.XGBClassifier(max_depth=8, n_estimators=450, learning_rate=0.2)]

for clf in xrange(len(classifiers)):
    classifiers[clf].fit(X_train, y_train)
    y_pred = classifiers[clf].predict(X_test)
    print str(names[clf])+" f1_score "+str(1000*f1_score(y_test, y_pred, average='micro'))

Nearest Neighbors f1_score 957.055214724
RBF SVM f1_score 990.797546012
xgb f1_score 990.797546012


In [44]:
#ensemble methodology to analyse performance

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score
clf2=SVC(gamma=0.4, C=7, decision_function_shape='ovr', tol=1e-4)
clf3=xgb.XGBClassifier(max_depth=8, n_estimators=400, learning_rate=0.2)
ensemble = VotingClassifier(estimators=[('svm', clf2), ('xgb', clf3)], voting='hard')
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)
print "f1_score "+str(1000*f1_score(y_test, y_pred, average='micro'))

f1_score 987.73006135


In [32]:
# creating prediction files for corresponding methodologies

clf3.fit(X_train,y_train)
submission_xgb = clf3.predict(test)

clf2.fit(X_train,y_train)
submission_svm = clf2.predict(test)

In [33]:
submission_ensemble = ensemble.predict(test)

In [34]:
submission_ensemble = pd.DataFrame(submission_ensemble)
submission_xgb = pd.DataFrame(submission_xgb)
submission_svm = pd.DataFrame(submission_svm)

In [35]:
# prediction.csv
submission_svm.to_csv('prediction.csv', header= False, index=False )